

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/SENTENCE_RELATIONS.ipynb)



# **Assess relationship between two sentences**

Evaluate the relationship between two sentences or text fragments to identify things such as contradictions, entailments and premises & hypotheses.

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

## 2. Start Spark Session

In [3]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


## 3. Select the model to use

In [4]:
#MODEL_NAME = 't5_small'
MODEL_NAME = 't5_base'

### 3.1 Select the task

The `T5 Transformer` model is able to perform 18 different tasks (ref.: [this paper](https://arxiv.org/abs/1910.10683)). To check the relations between sentences/texts, we can use the following tasks:

- `rte`: Recognizing, given two text fragments, whether the meaning of one text can be inferred (entailed) from the other or not.
- `mnli`: Classify for a hypothesis and premise whether they contradict or contradict each other or neither of both (3 classes).
- `qnli`: Classify whether the answer to a question can be deducted from an answer candidate.
- `cb`: Classify for a premise and a hypothesis whether they contradict each other or not (binary).
- `copa`: Classify for a question, premise, and 2 choices which choice the correct choice is (binary).
- `multirc`: Classify for a question, a paragraph of text, and an answer candidate, if the answer is correct (binary).

In [5]:
#TASK = 'rte'
#TASK = 'mnli'
#TASK = 'qnli'
#TASK = 'cb'
#TASK = 'copa'
TASK = 'multirc'

In [6]:
# Prefix to be used on the T5Transformer().setTask(<<prefix>>)
task_prefix = {
                'rte': 'rte sentence1:', 
                'mnli': 'mnli hypothesis:',
                'qnli': 'QNLI sentence1:', 
                'cb': 'cb hypothesis:', 
                'copa': 'copa choice1:', 
                'multirc': 'multirc questions:'    
            }

## 4 Examples to try on the model

In [7]:
text_lists = {
            'rte':      ["""
                        Kessler’s team conducted 60,643 interviews with adults in 14 countries.
                        Sentence 2: Kessler’s team interviewed more than 60,000 adults in 14 countries""",
                        """
                        Peter loves New York, it is his favorite city. 
                        Sentence 2: Peter loves new York.
                        """,
                        """
                        Recent report say Johnny makes he alot of money, he earned 10 million USD each year for the last 5 years. 
                        Sentence 2: Johnny is a millionare.
                        """,
                        """
                        Recent report say Johnny makes he alot of money, he earned 10 million USD each year for the last 5 years. 
                        Sentence 2: Johnny is a poor man.
                        """,
                        """
                        It was raining in England for the last 4 weeks. 
                        Sentence 2: England was very dry yesterday.
                        """],
            'mnli':     ["""
                        Recent report say Johnny makes he alot of money, he earned 10 million USD each year for the last 5 years.
                        premise: Johnny is a poor man.
                        """,
                        """
                        It rained in England the last 4 weeks.
                        premise: It was snowing in New York last week.
                        """],
            'qnli':     ["""
                        question: Where did Jebe die?
                        sentence: Ghenkis Khan recalled Subtai back to Mongolia soon afterward, and Jebe died on the road back to Samarkand 	
                        """,
                        """
                        question: What does Steve like to eat?	
                        sentence: Steve watches TV all day.
                        """],
            'cb':       ["""
                        hypothesis: Recent report say Johnny makes he alot of money, he earned 10 million USD each year for the last 5 years.
                        premise: Johnny is a poor man.
                        """],
            'copa':     ["""
                        choice1: He fell off the ladder    
                        choice2: He climbed up the lader       
                        premise: The man lost his balance on the ladder 
                        question: effect
                        """],
            'multirc':  ["""
                        questions: Why was Joey surprised the morning he woke up for breakfast?      
                        answer: There was a T-REX in his garden.      
                        paragraph:
                        Sent 1: Once upon a time, there was a squirrel named Joey.      
                        Sent 2: Joey loved to go outside and play with his cousin Jimmy.      
                        Sent 3: Joey and Jimmy played silly games together, and were always laughing.      
                        Sent 4: One day, Joey and Jimmy went swimming together 50 at their Aunt Julie’s pond.      
                        Sent 5: Joey woke up early in the morning to eat some food before they left.      
                        Sent 6: He couldn’t find anything to eat except for pie!      
                        Sent 7: Usually, Joey would eat cereal, fruit (a pear), or oatmeal for breakfast.      
                        Sent 8: After he ate, he and Jimmy went to the pond.      
                        Sent 9: On their way there they saw their friend Jack Rabbit.      
                        Sent 10: They dove into the water and swam for several hours.      
                        Sent 11: The sun was out, but the breeze was cold.      
                        Sent 12: Joey and Jimmy got out of the water and started walking home.      
                        Sent 13: Their fur was wet, and the breeze chilled them.      
                        Sent 14: When they got home, they dried off, and Jimmy put on his favorite purple shirt.      
                        Sent 15: Joey put on a blue shirt with red and green dots.      
                        Sent 16: The two squirrels ate some food that Joey’s mom, Jasmine, made and went off to bed.
                        """,
                        """
                        questions:  Why was Joey surprised the morning he woke up for breakfast?      
                        answer:There was only pie for breakfast.      
                        paragraph:
                        Sent 1: Once upon a time, there was a squirrel named Joey.      
                        Sent 2: Joey loved to go outside and play with his cousin Jimmy.      
                        Sent 3: Joey and Jimmy played silly games together, and were always laughing.      
                        Sent 4: One day, Joey and Jimmy went swimming together 50 at their Aunt Julie’s pond.      
                        Sent 5: Joey woke up early in the morning to eat some food before they left.      
                        Sent 6: He couldn’t find anything to eat except for pie!      
                        Sent 7: Usually, Joey would eat cereal, fruit (a pear), or oatmeal for breakfast.      
                        Sent 8: After he ate, he and Jimmy went to the pond.      
                        Sent 9: On their way there they saw their friend Jack Rabbit.      
                        Sent 10: They dove into the water and swam for several hours.      
                        Sent 11: The sun was out, but the breeze was cold.      
                        Sent 12: Joey and Jimmy got out of the water and started walking home.      
                        Sent 13: Their fur was wet, and the breeze chilled them.      
                        Sent 14: When they got home, they dried off, and Jimmy put on his favorite purple shirt.      
                        Sent 15: Joey put on a blue shirt with red and green dots.      
                        Sent 16: The two squirrels ate some food that Joey’s mom, Jasmine, made and went off to bed. 
                        """]}

## 5. Define the Spark NLP pipeline

In [8]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("documents")

t5 = T5Transformer() \
    .pretrained(MODEL_NAME) \
    .setTask(task_prefix[TASK])\
    .setMaxOutputLength(200)\
    .setInputCols(["documents"]) \
    .setOutputCol("T5")

pipeline = Pipeline(
    stages=[document_assembler, 
            t5])

t5_base download started this may take some time.
Approximate size to download 451.8 MB
[OK!]


## 6. Run the pipeline

In [9]:
# Fit on empty data frame (model is pretrained)
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = pipeline.fit(empty_df)

# Send example texts to spark data frame
text_df = spark.createDataFrame(pd.DataFrame({'text': text_lists[TASK]}))

# Predict with the Pipeline model
result = pipeline_model.transform(text_df)

# Create Light Pipeline
lmodel = LightPipeline(pipeline_model)

# Predict with then Ligh Pipeline model
res = lmodel.fullAnnotate(text_lists[TASK])

Before _validateStagesInputCols


## 7. Visualize the results

Using Light Pipeline:

In [10]:
for r in res:
    print(f"{r['documents'][0].result} => {r['T5'][0].result}")


                        questions: Why was Joey surprised the morning he woke up for breakfast?      
                        answer: There was a T-REX in his garden.      
                        paragraph:
                        Sent 1: Once upon a time, there was a squirrel named Joey.      
                        Sent 2: Joey loved to go outside and play with his cousin Jimmy.      
                        Sent 3: Joey and Jimmy played silly games together, and were always laughing.      
                        Sent 4: One day, Joey and Jimmy went swimming together 50 at their Aunt Julie’s pond.      
                        Sent 5: Joey woke up early in the morning to eat some food before they left.      
                        Sent 6: He couldn’t find anything to eat except for pie!      
                        Sent 7: Usually, Joey would eat cereal, fruit (a pear), or oatmeal for breakfast.      
                        Sent 8: After he ate, he and Jimmy went to the pond. 

Using pipeline model:

In [11]:
result.select('text', 'T5.result').show(truncate=80)

+--------------------------------------------------------------------------------+-------+
|                                                                            text| result|
+--------------------------------------------------------------------------------+-------+
|\n                        questions: Why was Joey surprised the morning he wo...|[False]|
|\n                        questions:  Why was Joey surprised the morning he w...| [True]|
+--------------------------------------------------------------------------------+-------+

